# Comparison performances DBMS / Ontology

## SPARQL

In [1]:
import owlready2 as owl
filename_onto_individuals = "../../src/ontology/final-archi-individuals.owl"
onto_individuals = owl.get_ontology(filename_onto_individuals).load()

In [3]:
def test_sparql():
    s, n, w, e = 42.69998105639557, 44.49946827276767, 0.20684058124504337, 2.6917189331881017

    near_airports = list(owl.default_world.sparql(
        f"""
            PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
            SELECT ?name ?iata ?icao ?latitude ?longitude ?altitude ?country
            WHERE {{
                ?Airport pie:AirportName ?name .
                ?Airport pie:AirportIATA ?iata .
                ?Airport pie:AirportICAOCode ?icao .
                ?Airport pie:AirportGPSLatitude ?latitude .
                ?Airport pie:AirportGPSLongitude ?longitude .
                ?Airport pie:AirportAltitude ?altitude .
                ?Airport pie:AirportCountry ?country .
                FILTER (?longitude > {w} && ?longitude < {e} 
                    &&  ?latitude > {s} && ?latitude < {n})

            }}
        """))
    fields = ['name', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'country']
    dict_airports_onto = [dict(zip(fields, airport_tuple)) for airport_tuple in near_airports]
    return dict_airports_onto

In [13]:
test_sparql()

[{'name': 'Carcassonne Airport',
  'iata': 'CCF',
  'icao': 'LFMK',
  'latitude': 43.215809,
  'longitude': 2.308544,
  'altitude': 433,
  'country': 'France'},
 {'name': 'Castres Mazamet Airport',
  'iata': 'DCM',
  'icao': 'LFCK',
  'latitude': 43.554958,
  'longitude': 2.290592,
  'altitude': 787,
  'country': 'France'},
 {'name': 'Toulouse Blagnac Airport',
  'iata': 'TLS',
  'icao': 'LFBO',
  'latitude': 43.634998,
  'longitude': 1.367778,
  'altitude': 499,
  'country': 'France'},
 {'name': 'Albi Le Sequestre Airport',
  'iata': 'LBI',
  'icao': 'LFCI',
  'latitude': 43.913269,
  'longitude': 2.11675,
  'altitude': 564,
  'country': 'France'},
 {'name': 'Agen La Garenne Airport',
  'iata': 'AGF',
  'icao': 'LFBA',
  'latitude': 44.174709,
  'longitude': 0.590619,
  'altitude': 204,
  'country': 'France'},
 {'name': 'Rodez Marcillac Airport',
  'iata': 'RDZ',
  'icao': 'LFCR',
  'latitude': 44.407501,
  'longitude': 2.483333,
  'altitude': 1910,
  'country': 'France'}]

## Pandas

In [11]:
import pandas as pd

all_airports = list(owl.default_world.sparql(
    f"""
        PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
        SELECT ?name ?iata ?icao ?latitude ?longitude ?altitude ?country
        WHERE {{
            ?Airport pie:AirportName ?name .
            ?Airport pie:AirportIATA ?iata .
            ?Airport pie:AirportICAOCode ?icao .
            ?Airport pie:AirportGPSLatitude ?latitude .
            ?Airport pie:AirportGPSLongitude ?longitude .
            ?Airport pie:AirportAltitude ?altitude .
            ?Airport pie:AirportCountry ?country .
        }}
    """))
dict_all_airports = [dict(zip(fields, airport_tuple)) for airport_tuple in all_airports]
df_all_airports = pd.DataFrame(dict_all_airports)

In [23]:
def test_pandas():
    s, n, w, e = 42.69998105639557, 44.49946827276767, 0.20684058124504337, 2.6917189331881017
    df_near_airpors = df_all_airports.loc[df_all_airports['longitude'].between(w, e) & df_all_airports['latitude'].between(s, n)]
    return df_near_airpors.to_dict('records')

test_pandas()

[{'name': 'Agen La Garenne Airport',
  'iata': 'AGF',
  'icao': 'LFBA',
  'latitude': 44.174709,
  'longitude': 0.590619,
  'altitude': 204,
  'country': 'France'},
 {'name': 'Albi Le Sequestre Airport',
  'iata': 'LBI',
  'icao': 'LFCI',
  'latitude': 43.913269,
  'longitude': 2.11675,
  'altitude': 564,
  'country': 'France'},
 {'name': 'Carcassonne Airport',
  'iata': 'CCF',
  'icao': 'LFMK',
  'latitude': 43.215809,
  'longitude': 2.308544,
  'altitude': 433,
  'country': 'France'},
 {'name': 'Castres Mazamet Airport',
  'iata': 'DCM',
  'icao': 'LFCK',
  'latitude': 43.554958,
  'longitude': 2.290592,
  'altitude': 787,
  'country': 'France'},
 {'name': 'Rodez Marcillac Airport',
  'iata': 'RDZ',
  'icao': 'LFCR',
  'latitude': 44.407501,
  'longitude': 2.483333,
  'altitude': 1910,
  'country': 'France'},
 {'name': 'Toulouse Blagnac Airport',
  'iata': 'TLS',
  'icao': 'LFBO',
  'latitude': 43.634998,
  'longitude': 1.367778,
  'altitude': 499,
  'country': 'France'}]

## Performance

In [25]:
%%timeit
test_sparql()

170 ms ± 22.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit
test_pandas()

2.06 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
